In [6]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torchvision import transforms

In [7]:
transform = transforms.Compose([transforms.Resize((512,512)),transforms.ToTensor()])

In [9]:
training_data = datasets.Flowers102(root = 'data', split = 'train', download = True, transform = transform)
testing_data = datasets.Flowers102(root = 'data', split = 'test', download = True, transform = transform)

100%|████████████████████████| 344862509/344862509 [01:26<00:00, 3990122.77it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|█████████████████████████████████████| 502/502 [00:00<00:00, 247652.39it/s]


100%|████████████████████████████████| 14989/14989 [00:00<00:00, 6835753.25it/s]
